In [20]:
import lightning as L
from torch.utils.data import DataLoader

from disent.dataset import DisentDataset
from disent.dataset.data import XYObjectData, Shapes3dData, SpritesData
from disent.dataset.transform import ToImgTensorF32
from disent.frameworks.vae import BetaVae
from disent.model import AutoEncoder
from disent.model.ae import DecoderConv64
from disent.model.ae import EncoderConv64
from disent.util import is_test_run  # you can ignore and remove this

# prepare the data
data = Shapes3dData(prepare=False)
data = SpritesData(prepare=True)

dataset = DisentDataset(data, transform=ToImgTensorF32())
dataloader = DataLoader(dataset=dataset, batch_size=4, shuffle=True, num_workers=10)

# create the pytorch lightning system
module: L.LightningModule = BetaVae(
    model=AutoEncoder(
        encoder=EncoderConv64(x_shape=data.x_shape, z_size=8, z_multiplier=2),
        decoder=DecoderConv64(x_shape=data.x_shape, z_size=8),
    ),
    cfg=BetaVae.cfg(optimizer="adam", optimizer_kwargs=dict(lr=1e-3), loss_reduction="mean_sum", beta=4),
)



In [26]:
# train the model
trainer = L.Trainer(logger=True, fast_dev_run=is_test_run(), max_epochs=3, )
                    # enable_checkpointing=True, default_root_dir='.data/checkpoints')
trainer.fit(module, dataloader)

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
/mnt/data/emanuele.marconato/miniconda3/envs/disent-py38/lib/python3.8/site-packages/lightning/pytorch/trainer/configuration_validator.py:70: PossibleUserWarning: You defined a `validation_step` but have no `val_dataloader`. Skipping val loop.
  rank_zero_warn(
Initializing distributed: GLOBAL_RANK: 0, MEMBER: 1/3
[W socket.cpp:426] [c10d] The server socket has failed to bind to [::]:55326 (errno: 98 - Address already in use).
/mnt/data/emanuele.marconato/miniconda3/envs/disent-py38/lib/python3.8/site-packages/lightning/pytorch/trainer/configuration_validator.py:70: PossibleUserWarning: You defined a `validation_step` but have no `val_dataloader`. Skipping val loop.
  rank_zero_warn(
Initializing distributed: GLOBAL_RANK: 1, MEMBER: 2/3
/mnt/data/emanuele.marconato/miniconda3/envs/disent-py38/lib/python3.8/site-packages/lightning/pytorc

ProcessRaisedException: 

-- Process 0 terminated with the following error:
Traceback (most recent call last):
  File "/mnt/data/emanuele.marconato/miniconda3/envs/disent-py38/lib/python3.8/site-packages/torch/multiprocessing/spawn.py", line 69, in _wrap
    fn(i, *args)
  File "/mnt/data/emanuele.marconato/miniconda3/envs/disent-py38/lib/python3.8/site-packages/lightning/pytorch/strategies/launchers/multiprocessing.py", line 147, in _wrapping_function
    results = function(*args, **kwargs)
  File "/mnt/data/emanuele.marconato/miniconda3/envs/disent-py38/lib/python3.8/site-packages/lightning/pytorch/trainer/trainer.py", line 570, in _fit_impl
    self._run(model, ckpt_path=ckpt_path)
  File "/mnt/data/emanuele.marconato/miniconda3/envs/disent-py38/lib/python3.8/site-packages/lightning/pytorch/trainer/trainer.py", line 933, in _run
    self.strategy.setup_environment()
  File "/mnt/data/emanuele.marconato/miniconda3/envs/disent-py38/lib/python3.8/site-packages/lightning/pytorch/strateg

In [4]:
from disent.metrics import metric_dci, metric_mig
from disent.util import is_test_run

get_repr = lambda x: module.encode(x.to(module.device))

# evaluate
{
    **metric_dci(
        dataset,
        get_repr,
        num_train=10 if is_test_run() else 1000,
        num_test=5 if is_test_run() else 500,
        boost_mode="sklearn",
    ),
    **metric_mig(dataset, get_repr, num_train=20 if is_test_run() else 2000),
}


{'dci.informativeness_train': 0.96025,
 'dci.informativeness_test': 0.24649999999999997,
 'dci.disentanglement': 0.02376654453393539,
 'dci.completeness': 0.02985560412256319,
 'mig.discrete_score': 0.035976604417420434}